In [5]:
import json
from tqdm import tqdm
from malaysian_sft import accept

In [14]:
jawi = []
with open('synthetic-jawi-conversation.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)

        try:
            s = l[1]['content_ms'].split()
            if len(s) < 250:
                continue

            if len(s) > 1000:
                continue

            if not accept(l[1]['content_ms']):
                continue      

            jawi.append(l[:2])

        except Exception as e:
            pass

system = [
    'you are a chatbot that always reply in jawi',
    'you are an assistant that always reply in jawi',
    'awak adalah chatbot yang sentiasa respond dalam jawi',
    'awak adalah pembantu AI yang reply in jawi',
]

system2 = [
    'respond in jawi, {text}',
    '{text}, reply in jawi',
    'reply in jawi\n\n{text}',
    '{text}\n\nrespond dlm jawi',
]

j = [j for j in jawi if j[0]['role'] != 'context']
j = sorted(j, key = lambda x: len(x[1]['content_ms']), reverse = True)

In [20]:
import random

data = []
for j_ in j:
    if random.randint(0, 1):
        s = None
        l = random.choice(system2).format(text = j_[0]['content_ms'])
    else:
        s = random.choice(system)
        l = j_[0]['content_ms']
        
    data.append({
        'prompt_input': s,
        'input': l,
        'output': j_[1]['content_jawi'],
    })

In [21]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

,prompt_input,input,output
0,None,Mengapa ahli politik Malaysia masih terlibat d...,ميمڠ بنر بهاوا ڤوليتيك ڤرقومان دان اڬام تله من...
1,None,Bagaimana Malaysia boleh terus bergantung kepa...,مليسيا تله لاما برڬنتوڠ كڤد بوروه اسيڠ، تراوتم...
2,you are an assistant that always reply in jawi,Bagaimanakah legasi skandal rasuah Najib Razak...,سكندل رسواه يڠ مليبتكن بكس ڤردان منتري مليسيا ...
3,awak adalah pembantu AI yang reply in jawi,Bagaimanakah legasi rasuah di bawah kepimpinan...,كڤيمڤينن نجيب رازق د مليسيا دري ٢٠٠٩ هيڠڬ ٢٠١٨...
4,None,reply in jawi\n\nMengapa badan kehakiman Malay...,بادن كهكيمن مليسيا تله مڠهادڤي تودوهن ممڤوليتي...


In [22]:
df.to_parquet('prepared-synthetic-jawi-conversation.parquet')

In [23]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-synthetic-jawi-conversation.parquet",
    path_in_repo='data/synthetic_jawi_conversation-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-synthetic-jawi-conversation.parquet:   0%|          | 0.00/217M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/85553928b10471c0d1162bce7c84105a93b6efb9', commit_message='Upload data/synthetic_jawi_conversation-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='85553928b10471c0d1162bce7c84105a93b6efb9', pr_url=None, pr_revision=None, pr_num=None)